In [1]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 近代数学とプログラミング (1)

無限の対象を扱います。プログラムが無限ループに陥ることがあるかもしれません。実行したセルが ``In [*]: `` の状態から戻らないときは、メニューの ``Kernel`` から ``Interrupt`` を選択して下さい。

## natural number: $\mathbb{Z}$

* [Natural number - Wikipedia](https://en.wikipedia.org/wiki/Natural_number)

* (1858&ndash;1932) [Giuseppe Peano - Wikipedia](https://en.wikipedia.org/wiki/Giuseppe_Peano)
  - [Peano axioms - Wikipedia](https://en.wikipedia.org/wiki/Peano_axioms)

### ペアノによる自然数の定義

1. $0$ is a natural number.
1. For every natural number $x$, $x = x$. That is, equality is *reflexive*. ([Reflexive relation - Wikipedia](https://en.wikipedia.org/wiki/Reflexive_relation))
1. For all natural numbers $x$ and $y$, if $x = y$, then $y = x$. That is, equality is *symmetric*. ([Symmetric relation - Wikipedia](https://en.wikipedia.org/wiki/Symmetric_relation))
1. For all natural numbers $x$, $y$ and $z$, if $x = y$ and $y = z$, then $x = z$. That is, equality is *transitive*. ([Transitive relation - Wikipedia](https://en.wikipedia.org/wiki/Transitive_relation))
1. For all $a$ and $b$, if $b$ is a natural number and $a = b$, then $a$ is also a natural number. That is, the natural numbers are closed under equality.

### 後者関数: $S$
* [Successor function - Wikipedia](https://en.wikipedia.org/wiki/Successor_function)

$$
  \begin{align}
    1 &= S(0) \\
    m + 0	 &= m \\
    m + S(n) &= S(m + n)
  \end{align}
$$

$$
  \begin{align}
    1+1 &= 1+S(0) = S(1+0) = S(1) = 2 \\
    1+2 &= 1+S(1) = S(1+1) = S(1+S(0)) = S(S(1+0)) = S(S(1)) = 3
  \end{align}
$$

1. For every natural number $n$, $S(n)$ is a natural number. That is, the natural numbers are closed under $S$.
1. For all natural numbers $m$ and $n$, $m = n$ if and only if $S(m) = S(n)$. That is, $S$ is an *injection*.
1. For every natural number $n$, $S(n) = 0$ is false. That is, there is no natural number whose successor is $0$.

後者関数 ``succ`` で自然数を生成する:
* 関数外では参照しないので、関数内で定義する。
* 関数の戻り値を ``return`` ではなく ``yield`` で返す。
  - 関数は生成子 (generator) を返す。
  - 関数 ``next()`` で繰り返して次の値を取り出せるようになる。

In [39]:
def natural_numbers():

    def succ(n):
        return n+1

    n = 0
    while True:
        yield n
        n = succ(n)

In [40]:
n= natural_numbers()
n

<generator object natural_numbers at 0x105578ba0>

In [41]:
next(n)

0

In [42]:
next(n), next(n), next(n)

(1, 2, 3)

In [43]:
for i in range(10):
    print(next(n))

4
5
6
7
8
9
10
11
12
13


In [44]:
for i in range(10):
    print(next(n))

14
15
16
17
18
19
20
21
22
23


In [45]:
for i in range(10):
    print(next(n))

24
25
26
27
28
29
30
31
32
33


### 匿名関数を使う

* [Anonymous function - Wikipedia](https://en.wikipedia.org/wiki/Anonymous_function)
  * [Lambda calculus - Wikipedia](https://en.wikipedia.org/wiki/Lambda_calculus)
    - [Church encoding - Wikipedia](https://en.wikipedia.org/wiki/Church_encoding)

* [Functional Programming HOWTO — Python 3.10.5 documentation](https://docs.python.org/3/howto/functional.html)

``lambda`` 式を使うと、変数に格納可能な匿名関数を定義することができる。``lambda`` 式は、関数に式を渡して、関数の実行時にその振る舞いを変えるときに使う。
* Pythonの匿名関数の本文は、ひとつの式しか持つことがず、その式の戻り値を返す。
* ``return`` は使えない。

In [46]:
def natural_numbers(succ = lambda n: n+1):
    n = 0
    while True:
        yield n
        n = succ(n)

In [47]:
n= natural_numbers()
n

<generator object natural_numbers at 0x105578f90>

In [48]:
{next(n) for i in range(10)}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [49]:
e = natural_numbers(succ = lambda n: n+2)
e

<generator object natural_numbers at 0x10557d2e0>

In [51]:
{next(e) for i in range(10)}

{0, 2, 4, 6, 8, 10, 12, 14, 16, 18}

課題-1) 上で定義した ``natural_numbers()`` を奇数を生成できるように改変せよ。

In [58]:
def natural_numbers(n=0, succ = lambda n: n+2):
    ### BEGIN SOLUTION
    while True:
        yield n
        n = succ(n)
    ### END SOLUTION

In [59]:
o = natural_numbers(n=1, succ = lambda n: n+2)
o

<generator object natural_numbers at 0x10557dcf0>

In [60]:
{next(o) for i in range(10)}

{1, 3, 5, 7, 9, 11, 13, 15, 17, 19}

---
以下、採点用のセルにつき編集できない:

In [ ]:
from nose.tools import assert_equal, assert_true

In [ ]:
assert_equal(next(natural_numbers(n=1, succ = lambda n: n+2)), 1)